In [6]:
import pandas as pd
import numpy as np

subset_method = "union"
correlation_type = "pearson"
data_category = pd.read_feather("../../all_data/xwas/univariate_correlations/correlations/categories/correlations_UrineBiochemistry.feather").to_dict()

In [ ]:
correlations_raw = pd.DataFrame(data_category).set_index(["dimension_1", "subdimension_1", "r2_1", "r2_std_1", "dimension_2", "subdimension_2", "r2_2", "r2_std_2"])
correlations_raw.columns = pd.MultiIndex.from_tuples(
    list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"]
)
correlations = correlations_raw[[(subset_method, correlation_type), (subset_method, "number_variables")]]
correlations.columns = ["correlation", "number_variables"]
correlations.reset_index(inplace=True)

MultiIndex([('union',          'pearson'),
            ('union', 'number_variables')],
           names=['subset_method', 'correlation_type'])

In [14]:
correlations

,dimension_1,subdimension_1,r2_1,r2_std_1,dimension_2,subdimension_2,r2_2,r2_std_2,correlation,number_variables
0,Abdomen,*,0.763341,0.002007,Abdomen,*,0.763341,0.002007,NaN,1.0
1,Abdomen,*,0.763341,0.002007,Abdomen,Liver,0.714937,0.002142,NaN,1.0
2,Abdomen,*,0.763341,0.002007,Abdomen,Pancreas,0.702804,0.002529,1.000000,2.0
3,Abdomen,*,0.763341,0.002007,Arterial,*,0.671142,0.006004,-1.000000,2.0
4,Abdomen,*,0.763341,0.002007,Arterial,Carotids,0.648211,0.005648,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...
1084,*instances23,*,0.900849,0.003993,PhysicalActivity,*,0.635326,0.001947,NaN,1.0
1085,*instances23,*,0.900849,0.003993,*,*,0.660751,0.001829,0.914919,4.0
1086,*instances23,*,0.900849,0.003993,*instances01,*,0.869851,0.003602,1.000000,2.0
1087,*instances23,*,0.900849,0.003993,*instances1.5x,*,0.635053,0.001931,NaN,1.0


In [15]:
table_correlations = correlations.pivot(
    index=["dimension_1", "subdimension_1"],
    columns=["dimension_2", "subdimension_2"],
    values="correlation",
)

customdata_list = []
for customdata_item in ["r2_1", "r2_std_1", "r2_2", "r2_std_2", "number_variables"]:
    customdata_list.append(
        correlations.pivot(
            index=["dimension_1", "subdimension_1"],
            columns=["dimension_2", "subdimension_2"],
            values=customdata_item,
        ).values
    )
stacked_customdata = list(map(list, np.dstack(customdata_list)))
